# PREDICCIÓN DE CASILLAS INE

In [1]:
# Carga de librerias
import pandas as pd
import numpy as np
import os
import glob

## DEFINICION DE FUNCIONES

In [2]:
# Características de archivo
def carac_file(name_file, estado):
    archivo   = pd.read_csv(name_file, low_memory = False)
    file      = archivo.query(f'ENTIDAD == {estado}')
    
    if "LISTA_NAL" in file:
        name_col = "LISTA_NAL"
    elif "LISTA" in file:
        name_col = "LISTA"
    else:
        name_col = "LISTA_NACIONAL"
    
    return file, name_col

In [3]:
# Filtrar datos del estado de guanajuato
def filtro(name_file, estado = 11, municipios = 46):
    file, columna = carac_file(name_file, estado)
    sumatoria     = np.array([], dtype = int) 
            
    for i in range(1, municipios + 1):
        suma_parcial = file.query(f"MUNICIPIO == {i}")[columna].sum()
        sumatoria    = np.append(sumatoria, suma_parcial)
  
    return  sumatoria 

In [4]:
# Condensación de datos (lista nominal por municipios)
def tabla_municipio(arc_names):
    table = pd.DataFrame()
    for name_file in arc_names:
        columna = filtro(name_file)
        table[name_file[:7]] = columna 
    return table

In [16]:
# Devuelve la tasa y los coeficientes del ajuste si se le indica 
def ajuste_tasa(data_mun, mes = 24, dev_coef = False):       # Febrero es el mes 24 de acuerdo 
    n_meses = np.arange(len(data_mun.columns))               # a la cantidad de meses descargados
    coeficientes = []
    tasa = []
    
    for i in range(len(data_mun.index)):
        lineal = np.polyfit(n_meses, data_mun.iloc[i].values.copy(),1)
        coeficientes.append(lineal)
        list_feb =  lineal[0] * mes + lineal[1]
        list_dic = data_mun[data_mun.columns[-1]][i]
        tasa.append(list_feb / list_dic)
        
    if dev_coef:
        return np.array(coeficientes)
    
    return tasa

In [14]:
# Determinación de número de casillas 
def casillas(name_file, tasa = np.ones(46), estado = 11):
    file, columna = carac_file(name_file, estado)
    
    secciones = []     
    for i in range(1,len(tasa)+1):
        secciones += list(((np.round(file.query(f"MUNICIPIO == {i}").sort_values('SECCION')[columna][:]) * tasa[i-1]) / 750) + 1)
    return  np.sum(np.array(secciones).astype(int))

## ESTRUCTURA 

In [7]:
# Ejecutar solo una vez!!!!
# Cambio de directorio
directorio = os.getcwd()+"\\Data INE"
os.chdir(directorio)
files = glob.glob('*.csv')

In [9]:
fdiciembre = files[-1] 

#### LLAMADA A FUNCIONES

In [21]:
data_municipio = tabla_municipio(files
tasa = ajuste_tasa(data_municipio)

print("**PROYECCIÓN DE CASILLAS**\n")
print(f"Número de casillas a instalar: {casillas(fdiciembre, tasa)}")

**PROYECCIÓN DE CASILLAS**

Número de casillas a instalar: 7630
